# Imports

In [5]:
import pandas as pd
from collections import Counter

# Prepare Training Data

In [3]:
df = pd.read_json('../data/MMHS150K_GT.json', orient='index', convert_axes=False, convert_dates=False, keep_default_dates=False) #

df.head(5)

,img_url,labels,tweet_url,tweet_text,labels_str
1114679353714016256,http://pbs.twimg.com/tweet_video_thumb/D3gi9MH...,"[4, 1, 3]",https://twitter.com/user/status/11146793537140...,@FriskDontMiss Nigga https://t.co/cAsaLWEpue,"[Religion, Racist, Homophobe]"
1063020048816660480,http://pbs.twimg.com/ext_tw_video_thumb/106301...,"[5, 5, 5]",https://twitter.com/user/status/10630200488166...,My horses are retarded https://t.co/HYhqc6d5WN,"[OtherHate, OtherHate, OtherHate]"
1108927368075374593,http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg,"[0, 0, 0]",https://twitter.com/user/status/11089273680753...,“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL S...,"[NotHate, NotHate, NotHate]"
1114558534635618305,http://pbs.twimg.com/ext_tw_video_thumb/111401...,"[1, 0, 0]",https://twitter.com/user/status/11145585346356...,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,"[Racist, NotHate, NotHate]"
1035252480215592966,http://pbs.twimg.com/media/Dl30pGIU8AAVGxO.jpg,"[1, 0, 1]",https://twitter.com/user/status/10352524802155...,“EVERYbody calling you Nigger now!” https://t....,"[Racist, NotHate, Racist]"


Make 1 Column for Classifier

In [7]:
dct = {0:0, 1:1, 2:1, 3:1, 4:1, 5:1}
df['labels'] = df['labels'].apply(lambda x: [*map(dct.get, x)])

In [8]:
df['majority_label'] = df.apply(lambda x: Counter(x['labels']).most_common()[0][0], axis=1)

Delete unnecessary columns

In [9]:
df = df.drop(columns=['img_url', 'tweet_url', 'labels_str', 'labels'])

Get Spark

In [10]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
import os

MAX_MEMORY = "5g"
spark = SparkSession.builder \
                    .appName('multi_class_text_classifiter')\
                    .config("spark.executor.memory", MAX_MEMORY) \
                    .config("spark.driver.memory", MAX_MEMORY) \
                    .getOrCreate()

22/11/29 14:42:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [11]:
# Diesen Abschnitt in eine Methode für Spark map auslagern.

import re

def clean_text(x):
    s = x.lower()
    s = re.sub(r'[^A-Za-z0-9 ]+', ' ', s)
    s = s.strip()
    
    return (s)

df['tweet_text_clean'] = df['tweet_text'].apply(clean_text)
df['tweet_text_clean']

1114679353714016256            friskdontmiss nigga https t co casalwepue
1063020048816660480         my horses are retarded https t co hyhqc6d5wn
1108927368075374593    nigga on ma momma youngboy be spitting real sh...
1114558534635618305    rt xxsugvngxx  i ran into this holy nigga toda...
1035252480215592966    everybody calling you nigger now  https t co 6...
                                             ...                        
1114170734472048640    svdate  gtconway3d i would just say hes donny ...
1110368198786846720    cheftime dev congrats my nigga keep on grindin...
1106941858540851200            my nigga big shitty https t co e0snjgbgh9
1105268309233188865    did she just say  my nigga  to rich   amp  she...
1114653514364530691    this nigga joe budden said thanos got a galact...
Name: tweet_text_clean, Length: 149823, dtype: object

In [12]:
from pyspark.sql import SQLContext

# sqlContext = SQLContext(spark)
df_small = df.sample(10000)
cleaned_df = spark.createDataFrame(df_small) #sqlContext

In [13]:
from pyspark.ml.feature import Word2Vec
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="tweet_text_clean", outputCol="tokens")
w2v = Word2Vec(vectorSize=300, minCount=0, inputCol="tokens", outputCol="features")
doc2vec_pipeline = Pipeline(stages=[tokenizer,w2v])
doc2vec_model = doc2vec_pipeline.fit(cleaned_df)
doc2vecs_df = doc2vec_model.transform(cleaned_df)

22/11/29 14:45:25 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/11/29 14:45:25 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [14]:
doc2vecs_df.show(1)

+--------------------+--------------+--------------------+--------------------+--------------------+
|          tweet_text|majority_label|    tweet_text_clean|              tokens|            features|
+--------------------+--------------+--------------------+--------------------+--------------------+
|Any movie with th...|             0|any movie with th...|[any, movie, with...|[0.00448822320686...|
+--------------------+--------------+--------------------+--------------------+--------------------+
only showing top 1 row



# Model Training

In [15]:
w2v_train_df, w2v_test_df = doc2vecs_df.randomSplit([0.8, 0.2])

In [16]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

rf_classifier = RandomForestClassifier(labelCol="majority_label", featuresCol="features")

rf_classifier_pipeline = Pipeline(stages=[rf_classifier])
rf_predictions = rf_classifier_pipeline.fit(w2v_train_df).transform(w2v_test_df)

rf_model_evaluator = MulticlassClassificationEvaluator(
    labelCol="majority_label", predictionCol="prediction", metricName="accuracy")

In [17]:
accuracy = rf_model_evaluator.evaluate(rf_predictions)
print("Accuracy = %g" % (accuracy))

Accuracy = 0.735917


In [18]:
from pyspark.ml.classification import LogisticRegression

lr_classifier = LogisticRegression(family="multinomial", labelCol="majority_label", featuresCol="features")

lr_classifier_pipeline = Pipeline(stages=[lr_classifier])
lr_predictions = lr_classifier_pipeline.fit(w2v_train_df).transform(w2v_test_df)

lr_model_evaluator = MulticlassClassificationEvaluator(
    labelCol="majority_label", predictionCol="prediction", metricName="accuracy")

In [19]:
accuracy = lr_model_evaluator.evaluate(lr_predictions)
print("Accuracy = %g" % (accuracy))

Accuracy = 0.743152


In [20]:
lr_predictions.show(10)

+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|          tweet_text|majority_label|    tweet_text_clean|              tokens|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|"Hoe ass nigga......|             0|hoe ass nigga  ht...|[hoe, ass, nigga,...|[-0.0088021091069...|[0.92616150267625...|[0.86439961978824...|       0.0|
|#EntlekICantDealW...|             1|entlekicantdealwi...|[entlekicantdealw...|[-0.0099941235412...|[1.35508797753429...|[0.93762444011813...|       0.0|
|#Ohio  Stop the d...|             0|ohio  stop the dr...|[ohio, , stop, th...|[-0.0476397339565...|[0.28989575573780...|[0.64101943171728...|       0.0|
|#she loves this d...|             0|she loves this dr...|[she, loves, this.